In [10]:
import numpy as np

In [ ]:
def fn(list):
    return np.log(np.sum([np.exp(x) for x in list]))

In [ ]:
a = [[1,2],[4,5],[7,8]]
a = np.array(a)

fn = min

# Apply fn along the dim=0 of a - column
np.apply_along_axis(fn, 0, a)

array([1, 2])